# Importing Libraries

In [25]:
import os
import zipfile

## GPU Detection

In [26]:
import subprocess
try:
    gpu_info = subprocess.check_output(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'])
    print("Detected GPU!!!")
    GPU_AVAILABLE = True
except Exception as e:
    print(f'No GPU Detected. Running on CPU!\n{e}')
    GPU_AVIALABLE = False

Detected GPU!!!


In [4]:
import dask.dataframe as dd
import dask.array as da
from dask_cuda import LocalCUDACluster
from dask.diagnostics import ProgressBar
from dask.distributed import Client

cluster = LocalCUDACluster()
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 2,Total memory: 30.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37707,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 2
Started: Just now,Total memory: 30.00 GiB
Comm: tcp://127.0.0.1:43397,Total threads: 1
Dashboard: http://127.0.0.1:44995/status,Memory: 15.00 GiB
Nanny: tcp://127.0.0.1:34719,


In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [28]:
try:
    import cudf
    import cupy as cp
    import dask_cudf
    GPU_MODE = torch.cuda.is_available()
except ImportError:
    cudf = None
    cp = None
    dask_cudf = None
    GPU_MODE = False

In [29]:
try:
    torch.multiprocessing.set_start_method('spawn', force=True)
    print("Set multiprocessing start method to 'spawn'.")
except RuntimeError:
    pass

Set multiprocessing start method to 'spawn'.


In [30]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    roc_auc_score,
    balanced_accuracy_score,
    matthews_corrcoef,
    confusion_matrix
)
from tqdm.auto import tqdm

# Configuration Settings

In [31]:
train_dir = "/kaggle/input/magnetic-data/magnetic_data/train"
dev_dir = "/kaggle/input/magnetic-data/magnetic_data/dev"
test_dir = "/kaggle/input/magnetic-data/magnetic_data/test"
scaler_params_path = "scaler_params.csv"

# CLIENT = Client(n_workers=4, threads_per_worker=2, memory_limit='4GB')
# print(f"Dask Dashboard Link: {CLIENT.dashboard_link}\n")

features = [
    'USFLUX_log', 'TOTPOT_log', 'PIL_LEN_log', 'MEANSHR', 'TOTFZ', 'EPSZ', 'R_VALUE',
    'USFLUX_log_roll_mean5', 'USFLUX_log_roll_std5', 'TOTPOT_log_diff1', 'MEANSHR_lag3'
]
target = 'label'
class_names = ['No Flare', 'C-Class', 'M-Class', 'X-Class']

batch_size = 256

num_cpus = os.cpu_count()
pin_memory = True
load_pre_trained_model = False


# Importing the Dataset

In [32]:
def encode_target(ddf, target_col_name):
    meta_dict = {col: ddf[col].dtype for col in ddf.columns if col != target_col_name}
    meta_dict['flare_level'] = np.int8
    
    def encode_partition(df):
        df['flare_level'] = 0
        df.loc[df[target_col_name].str.startswith('C'), 'flare_level'] = 1
        df.loc[df[target_col_name].str.startswith('M'), 'flare_level'] = 2
        df.loc[df[target_col_name].str.startswith('X'), 'flare_level'] = 3
        df['flare_level'] = df['flare_level'].astype(np.int8)
        return df.drop(columns=[target_col_name])
    
    meta_df = pd.DataFrame({col: pd.Series(dtype=dtype) for col, dtype in meta_dict.items()})
    
    ddf = ddf.map_partitions(encode_partition, meta=meta_df)
    
    return ddf

In [33]:
def Index(ddf):
    ddf['safe_index'] = ddf['record_id'].astype(int) * 60 + ddf['seq_id'].astype(int)
    ddf = ddf.map_partitions(lambda df: df.sort_values("safe_index"))
    ddf = ddf.set_index("safe_index", sorted=False, compute=True)
    return ddf

In [34]:
# train_dir = "../data/processed/magnetic_data/train/"
ddf_train = dask_cudf.read_parquet(train_dir)
# if 'index' in ddf_train.columns:
#     ddf_train = ddf_train.drop(columns=['index'])
ddf_train = Index(ddf_train)
ddf_train = encode_target(ddf_train, target)
print(f"Indexed ddf_train: {ddf_train.index.is_monotonic_increasing.compute()}")

2025-11-24 07:01:15,008 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 98f34a03f0697fdc39c981d7bb28b9d2 initialized by task ('shuffle-transfer-98f34a03f0697fdc39c981d7bb28b9d2', 2) executed on worker tcp://127.0.0.1:43397
2025-11-24 07:01:29,016 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 98f34a03f0697fdc39c981d7bb28b9d2 deactivated due to stimulus 'task-finished-1763967689.011179'


Indexed ddf_train: True


In [35]:
# train_dir = "../data/processed/magnetic_data/train/"
ddf_dev = dask_cudf.read_parquet(dev_dir)
ddf_dev = Index(ddf_dev)
ddf_dev = encode_target(ddf_dev, target)
print(f"Indexed ddf_dev: {ddf_dev.index.is_monotonic_increasing.compute()}")

# X_train = ddf_train[features].to_dask_array().compute_chunk_sizes()
# y_train = ddf_train['flare_level'].to_dask_array().compute_chunk_sizes()

2025-11-24 07:01:29,939 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 653e244e4651b24c2714797d4a8a2022 initialized by task ('shuffle-transfer-653e244e4651b24c2714797d4a8a2022', 3) executed on worker tcp://127.0.0.1:38977
2025-11-24 07:01:30,182 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 653e244e4651b24c2714797d4a8a2022 deactivated due to stimulus 'task-finished-1763967690.1805036'


Indexed ddf_dev: True


In [36]:
# test_dir = "../data/processed/magnetic_data/test/"
ddf_test = dask_cudf.read_parquet(test_dir)
ddf_test = Index(ddf_test)
ddf_test = encode_target(ddf_test, target)
print(f"Indexed ddf_test: {ddf_test.index.is_monotonic_increasing.compute()}")

# X_test = ddf_test[features].to_dask_array().compute_chunk_sizes()
# y_test = ddf_test['flare_level'].to_dask_array().compute_chunk_sizes()

2025-11-24 07:01:31,532 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 59bfa0fca3ee7ce3dfd65ae2befc0cd8 initialized by task ('shuffle-transfer-59bfa0fca3ee7ce3dfd65ae2befc0cd8', 3) executed on worker tcp://127.0.0.1:38977
2025-11-24 07:01:31,802 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 59bfa0fca3ee7ce3dfd65ae2befc0cd8 deactivated due to stimulus 'task-finished-1763967691.8014262'


Indexed ddf_test: True


In [37]:
df_gpu = ddf_train[['flare_level']].compute()  # Bring to a single CuDF on GPU
class_counts_multi = df_gpu['flare_level'].value_counts()
print(f"Multiclass Distribution:\n{class_counts_multi}")
print("Note the mapping: 0=No Flare, 1=C-Class, 2=M-Class, 3=X-Class\n")
class_counts = class_counts_multi.sort_index()
total = class_counts.sum()
class_weights = total / (len(class_counts) * class_counts)
class_weights_tensor = torch.tensor(class_weights.values, dtype=torch.float32).to('cuda')

2025-11-24 07:01:32,327 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 98f34a03f0697fdc39c981d7bb28b9d2 initialized by task ('shuffle-transfer-98f34a03f0697fdc39c981d7bb28b9d2', 3) executed on worker tcp://127.0.0.1:38977
2025-11-24 07:01:43,629 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 98f34a03f0697fdc39c981d7bb28b9d2 deactivated due to stimulus 'task-finished-1763967703.6277936'


Multiclass Distribution:
flare_level
0    11042040
1     1236240
2      227100
3       22620
Name: count, dtype: int64
Note the mapping: 0=No Flare, 1=C-Class, 2=M-Class, 3=X-Class



# Feature Scaling and Normalization

In [38]:
aggregations = {col: ['mean', 'std'] for col in features if col not in ['record_id', 'seq_id']}
scaler_params_df = (
    ddf_train.map_partitions(
        lambda gdf: gdf.groupby("seq_id").agg(aggregations)
    ).compute()
)

scaler_params_df.columns = ['_'.join(col).strip() for col in scaler_params_df.columns.values]
scaler_params_df.to_csv(scaler_params_path)

2025-11-24 07:01:44,674 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 98f34a03f0697fdc39c981d7bb28b9d2 initialized by task ('shuffle-transfer-98f34a03f0697fdc39c981d7bb28b9d2', 2) executed on worker tcp://127.0.0.1:43397
2025-11-24 07:01:55,332 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 98f34a03f0697fdc39c981d7bb28b9d2 deactivated due to stimulus 'task-finished-1763967715.3306108'


In [39]:
def timeseries_scaling(ddf, scaler_params_path, features):
    if isinstance(ddf, dd.DataFrame):
        ddf = dask_cudf.from_cudf(ddf.compute(), npartitions=ddf.npartitions)

    scaler_params = cudf.read_csv(scaler_params_path)
    scaler_params_ddf = dask_cudf.from_cudf(scaler_params, npartitions=1)

    ddf_scaled = ddf.merge(scaler_params_ddf, on="seq_id", how="left")

    for col in features:
        mean_col = f"{col}_mean"
        std_col = f"{col}_std"
        ddf_scaled[col] = (ddf_scaled[col] - ddf_scaled[mean_col]) / (ddf_scaled[std_col] + 1e-7)

    return ddf_scaled.persist()

In [ ]:
ddf_train = timeseries_scaling(ddf_train, scaler_params_path, features).persist()
ddf_dev = timeseries_scaling(ddf_dev, scaler_params_path, features).persist()
ddf_test = timeseries_scaling(ddf_test, scaler_params_path, features).persist()
# ddf_train.head(100)

In [19]:
!nvidia-smi

Sun Nov  9 21:27:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   75C    P0             34W /   70W |    9277MiB /  15360MiB |      2%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# DataLoader

In [41]:
class TimeSeriesDataset(Dataset):
    def __init__(self, ddf, features=features, target="flare_level", chunk_size=10000):
        self.features_col = features
        self.target = target
        seq_length = 60
        num_features = len(features)
        
        unique_ids_series = ddf["record_id"].unique().compute()
        
        if hasattr(unique_ids_series, 'to_arrow'):
            record_list = unique_ids_series.to_arrow().to_pylist()
        elif hasattr(unique_ids_series, 'to_pandas'):
            record_list = unique_ids_series.to_pandas().tolist()
        else:
            record_list = list(unique_ids_series)
        
        num_records = len(record_list)        
        self.X = []
        self.y = []
        
        num_chunks = (num_records + chunk_size - 1) // chunk_size
        
        for chunk_idx in range(num_chunks):
            start_idx = chunk_idx * chunk_size
            end_idx = min(start_idx + chunk_size, num_records)
            chunk_records = record_list[start_idx:end_idx]
            
            try:
                chunk_ddf = ddf[ddf["record_id"].isin(chunk_records)]
                chunk_df = chunk_ddf.compute()
                
                if hasattr(chunk_df, 'to_pandas'):
                    chunk_df = chunk_df.to_pandas()
                
                grouped = chunk_df.groupby("record_id")
                
                for record_id, seq_df in grouped:
                    # actual_len = len(seq_df)
                    
                    # if actual_len != seq_length and len(self.X) < 5:
                    #     print(f"\t\tRecord {record_id}: {actual_len} rows (expected {seq_length})")
                    
                    # if actual_len >= seq_length:
                    #     seq_df = seq_df.iloc[:seq_length]
                    # else:
                    #     continue
                    
                    try:
                        features_array = seq_df[self.features_col].values.astype(np.float32)
                        target_value = seq_df[self.target].mode().iloc[0]
                        
                        self.X.append(features_array)
                        self.y.append(target_value)
                        
                    except Exception as e:
                        if len(self.X) < 3:
                            print(f"ERROR: {e}")
                        continue
                
                del chunk_df, chunk_ddf
                
            except Exception as e:
                print(f"CHUNK ERROR: {e}")
                continue
        
        if len(self.X) == 0:
            raise ValueError("No sequences loaded! Check preprocessing pipeline.")
        
        self.X = np.array(self.X, dtype=np.float32)
        self.y = np.array(self.y, dtype=np.int64)

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return (
            torch.from_numpy(self.X[idx]),
            torch.tensor(self.y[idx], dtype=torch.long)
        )

In [42]:
def make_dataloader(dataset, batch_size=64, shuffle=True):
    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=0,
        pin_memory=not GPU_MODE,
        prefetch_factor=None
    )

In [ ]:
train_dataset = TimeSeriesDataset(ddf_train.persist())
train = make_dataloader(train_dataset)

dev_dataset = TimeSeriesDataset(ddf_dev.persist())
dev = make_dataloader(dev_dataset)

test_dataset = TimeSeriesDataset(ddf_test.persist())
test = make_dataloader(test_dataset)

# Attentive RNN

In [44]:
class AttentiveRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, attention = False, bidirectional = False):
        super().__init__()

        self.hidden_size = hidden_size
        if attention in ["dot", "concat", None]:
            self.attention = attention
        else:
            raise ValueError("attention can only be 'dot' or 'concat'!!!")
        self.num_directions = 2 if bidirectional else 1

        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first = True, bidirectional = bidirectional)
        self.concat = nn.Linear(hidden_size * self.num_directions, 1)

        self.fc = nn.Linear(hidden_size * self.num_directions, num_classes)
        
    def forward(self, X):
        # X shape: (batch_size, seq_len, input_size)
        batch_size, seq_len, input_size = X.shape
        rnn_out, h_n = self.rnn(X)
        # rnn_output: (batch_size, seq_len, hidden_size * num_directions)
        # h_n: (num_layers * num_directions, batch_size, hidden_size)
        score = None
        if self.attention == "concat":
            atten_input = rnn_out.reshape(-1, self.hidden_size * self.num_directions)     # (batch_size * seq-Len, hidden_size * num_directons)
            score = self.concat(atten_input).view(batch_size, -1)                         # (batch_size, seq_len)
        
        elif self.attention == "dot":
            
            if self.num_directions == 2:
                last_hidden = torch.cat((h_n[-2,:,:], h_n[-1,:,:]), dim=1)
            else:
                last_hidden = h_n[-1,:,:]
        
            query = last_hidden.unsqueeze(1)                                              # (batch_size, 1, hidden_size * num_directions)
            score = torch.bmm(query, rnn_out.transpose(1,2)).squeeze(1)                   # (batch_size, seq_len)
        
        context = None
        attn_weights = None
        
        if score is not None:
            attn_weights = F.softmax(score, dim=1)                                       # (batch_size, seq_len)
            context = torch.bmm(attn_weights.unsqueeze(1), rnn_out)                       # (batch_size, 1, hidden_size * num_directions)
            context = context.squeeze(1)                                                  # (batch_size, hidden_size * num_directions)
        else:
            if self.num_directions == 2:
                context = torch.cat((h_n[-2,:,:], h_n[-1,:,:]), dim=1)
            else:
                context = h_n[-1,:,:]

        output = self.fc(context)
        
        return output, attn_weights

# Attentive LSTM

In [45]:
class AttentiveLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, attention=None, bidirectional=False):
        super().__init__()

        self.hidden_size = hidden_size
        if attention in ["dot", "concat", None]:
            self.attention = attention
        else:
            raise ValueError("attention can only be 'dot', 'concat', or None!!!")
        
        self.num_directions = 2 if bidirectional else 1

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=bidirectional)
        
        if self.attention == "concat":
            self.attention_layer = nn.Linear(hidden_size * self.num_directions, 1)

        self.fc = nn.Linear(hidden_size * self.num_directions, num_classes)
        
    def forward(self, X):
        # X shape: (batch_size, seq_len, input_size)
        batch_size, seq_len, _ = X.shape
        
        lstm_out, (h_n, c_n) = self.lstm(X)
        # lstm_out: (batch_size, seq_len, hidden_size * num_directions)
        # h_n: (num_layers * num_directions, batch_size, hidden_size)
        # c_n: (num_layers * num_directions, batch_size, hidden_size)
        
        score = None
        if self.attention == "concat":
            atten_input = lstm_out.reshape(-1, self.hidden_size * self.num_directions)
            score = self.attention_layer(atten_input).view(batch_size, -1)
        
        elif self.attention == "dot":
            if self.num_directions == 2:
                last_hidden = torch.cat((h_n[-2,:,:], h_n[-1,:,:]), dim=1)
            else:
                last_hidden = h_n[-1,:,:]
        
            query = last_hidden.unsqueeze(1)
            score = torch.bmm(query, lstm_out.transpose(1,2)).squeeze(1)
        
        context = None
        attn_weights = None
        
        if self.attention is not None:
            attn_weights = F.softmax(score, dim=1)
            context = torch.bmm(attn_weights.unsqueeze(1), lstm_out)
            context = context.squeeze(1)
        else:
            if self.num_directions == 2:
                context = torch.cat((h_n[-2,:,:], h_n[-1,:,:]), dim=1)
            else:
                context = h_n[-1,:,:]

        output = self.fc(context)
        
        return output, attn_weights

# Training and Evaluation Functions

In [46]:
def training(model, device, criterion, optimizer, train_loader, train_epochs=10, pre_epochs=0):
    print(f"Starting training for {train_epochs} epochs on {device}.")
    train_start = time.time()
    epoch_pbar = tqdm(range(train_epochs), desc="Training Progress", unit="epoch")
    
    for epoch in epoch_pbar:
        model.train()
        total_loss = 0.0
        all_predictions = []
        all_targets = []
        
        for features, labels in train_loader:
            features = features.to(device) 
            labels = labels.to(device)

            optimizer.zero_grad()
            logits, _ = model(features)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            _, predicted_classes = torch.max(logits, 1)
            all_predictions.extend(predicted_classes.cpu().numpy())
            all_targets.extend(labels.cpu().numpy())

        avg_loss = total_loss / len(train_loader)
        accuracy = accuracy_score(all_targets, all_predictions)
        
        print(f"Epoch {pre_epochs + epoch + 1} Summary -> Avg. Loss: {avg_loss:.4f}, Accuracy: {accuracy*100:.2f}%")

    total_seconds = time.time() - train_start
    hours, rem = divmod(total_seconds, 3600)
    minutes, seconds = divmod(rem, 60)
    print(f"\nTotal Train Time: {int(hours)}h {int(minutes)}m {int(seconds)}s")

In [47]:
def eval_detailed(model, device, loader, report_title="Evaluation"):
    model.eval()
    all_targets, all_predictions, all_probas = [], [], []

    with torch.no_grad():
        for features, labels in tqdm(loader, desc=f"Evaluating ({report_title})", leave=False):
            features, labels = features.to(device), labels.to(device)
            logits, _ = model(features)
            probas = F.softmax(logits, dim=1)
            preds = torch.argmax(probas, dim=1)

            all_targets.extend(labels.cpu().numpy())
            all_predictions.extend(preds.cpu().numpy())
            all_probas.extend(probas.cpu().numpy())

    y_true = np.array(all_targets)
    y_pred = np.array(all_predictions)
    y_pred_proba = np.array(all_probas)

    print(f"\n================= {report_title} =================")
    print("Classification Report:\n")
    print(classification_report(
        y_true,
        y_pred,
        target_names=class_names,
        digits=4,
        zero_division=0
    ))

    try:
        roc_auc = roc_auc_score(y_true, y_pred_proba, multi_class='ovr', average='weighted')
    except ValueError:
        roc_auc = float("nan")

    metrics = {
        "balanced_accuracy": balanced_accuracy_score(y_true, y_pred),
        "roc_auc_weighted": roc_auc,
        "mcc": matthews_corrcoef(y_true, y_pred),
        "f1_macro": f1_score(y_true, y_pred, average='macro', zero_division=0),
        "f1_weighted": f1_score(y_true, y_pred, average='weighted', zero_division=0)
    }

    cm = confusion_matrix(y_true, y_pred, normalize='true')
    print("\nNormalized Confusion Matrix:\n", cm)

    print("\nKey Metrics Summary:")
    for k, v in metrics.items():
        print(f"  {k:20s}: {v:.4f}")

    return metrics


In [25]:
def save_model_as_zip(model, model_name, save_dir="/kaggle/working"):
    # File paths
    model_path = os.path.join(save_dir, f"{model_name}.pth")
    zip_path = os.path.join(save_dir, f"{model_name}.zip")

    # Save model
    torch.save(model.state_dict(), model_path)

    # Zip it
    with zipfile.ZipFile(zip_path, 'w', compression=zipfile.ZIP_DEFLATED) as zipf:
        zipf.write(model_path, arcname=f"{model_name}.pth")

    # Remove unzipped .pth file
    os.remove(model_path)

    # Get zipped size in MB
    size_bytes = os.path.getsize(zip_path)
    size_mb = size_bytes / (1024 ** 2)

    print(f"{model_name}.zip saved at {zip_path} | Size: {size_mb:.2f} MB")

# Experimenter

In [28]:
experiments = []
model_types = ["RNN", "LSTM"]
attention_types = [None, "dot", "concat"]
directionality = [False, True]

for model_t in model_types:
    for att_t in attention_types:
        for is_bi in directionality:
            # Create a descriptive name for the run
            direction_str = "Bi" if is_bi else "Uni"
            attention_str = att_t if att_t is not None else "NoAtt"
            run_name = f"{model_t}_{direction_str}_{attention_str}"
            
            config = {
                "run_name": run_name,
                "model_type": model_t,
                "hidden_size": 128,
                "num_layers": 2,
                "attention": att_t,
                "bidirectional": is_bi,
                "lr": 0.001,
                "epochs": 60
            }
            experiments.append(config)

In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

results_log = []
total_start_time = time.time()

for i, config in enumerate(experiments):
    print(f"\n{'='*30} Experiment {i+1}/{len(experiments)}: {config['run_name']} {'='*30}")
    
    experiment_start_time = time.time()
    
    if config["model_type"] == "RNN":
        model = AttentiveRNN(
            input_size=len(features),
            hidden_size=config["hidden_size"],
            num_layers=config["num_layers"],
            num_classes=len(class_names),
            attention=config["attention"],
            bidirectional=config["bidirectional"]
        )
    else: # LSTM
        model = AttentiveLSTM(
            input_size=len(features),
            hidden_size=config["hidden_size"],
            num_classes=len(class_names),
            num_layers=config["num_layers"],
            attention=config["attention"],
            bidirectional=config["bidirectional"]
        )
    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs via DataParallel!")
        model = nn.DataParallel(model)
    model.to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"])
    criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
    
    print(f"Training model: {config['run_name']} for {config['epochs']} epochs...")
    training(model, device, criterion, optimizer, train, train_epochs=config["epochs"])
    
    print(f"Evaluating model: {config['run_name']}...")
    eval_metrics = eval_detailed(model, device, dev, report_title=config['run_name'])
    
    experiment_time = time.time() - experiment_start_time
    save_model_as_zip(model, model_name=config['run_name'])
    
    run_result = {
        "run_name": config["run_name"],
        "model_type": config["model_type"],
        "num_layers": config["num_layers"],
        "hidden_size": config["hidden_size"],
        "attention": str(config["attention"]),
        "bidirectional": config["bidirectional"],
        **eval_metrics,
        "training_time_min": experiment_time / 60
    }
    results_log.append(run_result)
    
    pd.DataFrame(results_log).to_csv("experiment_report_intermediate.csv", index=False)


print(f"\n{'='*30} FINAL EXPERIMENT REPORT {'='*30}")
total_run_time = time.time() - total_start_time
print(f"Total experiment time: {total_run_time/60:.2f} minutes")

results_df = pd.DataFrame(results_log)
results_df = results_df.sort_values(by="mcc", ascending=False).reset_index(drop=True)

pd.set_option('display.float_format', '{:.4f}'.format)
display(results_df)

results_df.to_csv("experiment_report_final.csv", index=False)


============================== Experiment 1/12: RNN_Uni_NoAtt ==============================
Using 2 GPUs via DataParallel!
Training model: RNN_Uni_NoAtt for 60 epochs...
Starting training for 60 epochs on cuda.


Training Progress:   0%|          | 0/60 [00:00<?, ?epoch/s]

Epoch 1 Summary -> Avg. Loss: 0.8937, Accuracy: 74.97%
Epoch 2 Summary -> Avg. Loss: 0.8696, Accuracy: 74.98%
Epoch 3 Summary -> Avg. Loss: 0.8531, Accuracy: 74.35%
Epoch 4 Summary -> Avg. Loss: 0.8517, Accuracy: 74.64%
Epoch 5 Summary -> Avg. Loss: 0.8438, Accuracy: 74.53%
Epoch 6 Summary -> Avg. Loss: 0.8540, Accuracy: 74.18%
Epoch 7 Summary -> Avg. Loss: 0.9433, Accuracy: 72.51%
Epoch 8 Summary -> Avg. Loss: 0.8693, Accuracy: 75.24%
Epoch 9 Summary -> Avg. Loss: 0.8577, Accuracy: 74.73%
Epoch 10 Summary -> Avg. Loss: 0.8692, Accuracy: 73.64%
Epoch 11 Summary -> Avg. Loss: 0.8580, Accuracy: 73.41%
Epoch 12 Summary -> Avg. Loss: 0.8534, Accuracy: 74.13%
Epoch 13 Summary -> Avg. Loss: 0.8591, Accuracy: 74.06%
Epoch 14 Summary -> Avg. Loss: 0.8486, Accuracy: 75.31%
Epoch 15 Summary -> Avg. Loss: 0.8425, Accuracy: 74.59%
Epoch 16 Summary -> Avg. Loss: 0.8526, Accuracy: 75.08%
Epoch 17 Summary -> Avg. Loss: 0.8467, Accuracy: 75.10%
Epoch 18 Summary -> Avg. Loss: 0.8455, Accuracy: 74.53%
E

Evaluating (RNN_Uni_NoAtt):   0%|          | 0/34 [00:00<?, ?it/s]


================= RNN_Uni_NoAtt =================
Classification Report:

              precision    recall  f1-score   support

    No Flare     0.9781    0.8072    0.8845      1878
     C-Class     0.2725    0.6619    0.3861       210
     M-Class     0.2429    0.4359    0.3119        39
     X-Class     0.0000    0.0000    0.0000         4

    accuracy                         0.7846      2131
   macro avg     0.3734    0.4763    0.3956      2131
weighted avg     0.8932    0.7846    0.8232      2131


Normalized Confusion Matrix:
 [[8.07241747e-01 1.85303514e-01 6.92225772e-03 5.32481363e-04]
 [1.57142857e-01 6.61904762e-01 1.80952381e-01 0.00000000e+00]
 [2.56410256e-02 5.38461538e-01 4.35897436e-01 0.00000000e+00]
 [0.00000000e+00 5.00000000e-01 5.00000000e-01 0.00000000e+00]]

Key Metrics Summary:
  balanced_accuracy   : 0.4763
  roc_auc_weighted    : 0.9026
  mcc                 : 0.4025
  f1_macro            : 0.3956
  f1_weighted         : 0.8232
RNN_Uni_NoAtt.zip saved at /k

...

Training Progress:   0%|          | 0/60 [00:00<?, ?epoch/s]

Epoch 1 Summary -> Avg. Loss: 0.8318, Accuracy: 75.57%
Epoch 2 Summary -> Avg. Loss: 0.7688, Accuracy: 76.58%
Epoch 3 Summary -> Avg. Loss: 0.7331, Accuracy: 76.80%
Epoch 4 Summary -> Avg. Loss: 0.7099, Accuracy: 77.12%
Epoch 5 Summary -> Avg. Loss: 0.6942, Accuracy: 77.49%
Epoch 6 Summary -> Avg. Loss: 0.6689, Accuracy: 77.89%
Epoch 7 Summary -> Avg. Loss: 0.6677, Accuracy: 77.90%
Epoch 8 Summary -> Avg. Loss: 0.6411, Accuracy: 78.64%
Epoch 9 Summary -> Avg. Loss: 0.6162, Accuracy: 78.99%
Epoch 10 Summary -> Avg. Loss: 0.6033, Accuracy: 79.03%
Epoch 11 Summary -> Avg. Loss: 0.5778, Accuracy: 79.39%
Epoch 12 Summary -> Avg. Loss: 0.5657, Accuracy: 79.99%
Epoch 13 Summary -> Avg. Loss: 0.5557, Accuracy: 80.21%
Epoch 14 Summary -> Avg. Loss: 0.5325, Accuracy: 80.40%
Epoch 15 Summary -> Avg. Loss: 0.5165, Accuracy: 80.90%
Epoch 16 Summary -> Avg. Loss: 0.4881, Accuracy: 81.36%
Epoch 17 Summary -> Avg. Loss: 0.5015, Accuracy: 81.26%
Epoch 18 Summary -> Avg. Loss: 0.4611, Accuracy: 82.14%
E

Evaluating (LSTM_Bi_concat):   0%|          | 0/34 [00:00<?, ?it/s]


================= LSTM_Bi_concat =================
Classification Report:

              precision    recall  f1-score   support

    No Flare     0.9929    0.8962    0.9421      1878
     C-Class     0.5179    0.8952    0.6562       210
     M-Class     0.5217    0.9231    0.6667        39
     X-Class     0.7500    0.7500    0.7500         4

    accuracy                         0.8963      2131
   macro avg     0.6956    0.8661    0.7537      2131
weighted avg     0.9370    0.8963    0.9085      2131


Normalized Confusion Matrix:
 [[0.89616613 0.09158679 0.01224707 0.        ]
 [0.05714286 0.8952381  0.04285714 0.0047619 ]
 [0.         0.07692308 0.92307692 0.        ]
 [0.         0.         0.25       0.75      ]]

Key Metrics Summary:
  balanced_accuracy   : 0.8661
  roc_auc_weighted    : 0.9693
  mcc                 : 0.6634
  f1_macro            : 0.7537
  f1_weighted         : 0.9085
LSTM_Bi_concat.zip saved at /kaggle/working/LSTM_Bi_concat.zip | Size: 1.92 MB

============

,run_name,model_type,num_layers,hidden_size,attention,bidirectional,balanced_accuracy,roc_auc_weighted,mcc,f1_macro,f1_weighted,training_time_min
0,LSTM_Bi_dot,LSTM,2,128,dot,True,0.9346,0.9747,0.7080,0.7845,0.9254,61.5377
1,LSTM_Bi_concat,LSTM,2,128,concat,True,0.8661,0.9693,0.6634,0.7537,0.9085,61.6548
2,LSTM_Uni_dot,LSTM,2,128,dot,False,0.8893,0.9476,0.5587,0.6992,0.8675,33.1952
3,LSTM_Uni_concat,LSTM,2,128,concat,False,0.7513,0.9284,0.4674,0.5527,0.8345,33.3580
4,LSTM_Uni_NoAtt,LSTM,2,128,None,False,0.7765,0.9233,0.4628,0.5684,0.8285,31.1556
5,LSTM_Bi_NoAtt,LSTM,2,128,None,True,0.7822,0.9213,0.4318,0.4733,0.8261,59.8036
6,RNN_Uni_NoAtt,RNN,2,128,None,False,0.4763,0.9026,0.4025,0.3956,0.8232,23.5624
7,RNN_Bi_dot,RNN,2,128,dot,True,0.7576,0.9120,0.3934,0.4626,0.8251,30.2376
8,RNN_Uni_concat,RNN,2,128,concat,False,0.6985,0.9079,0.3929,0.4516,0.8077,25.8786
9,RNN_Bi_concat,RNN,2,128,concat,True,0.6332,0.9037,0.3757,0.4483,0.7757,28.7604


# Train Funrther

In [30]:
def unzip_and_load_model(model, model_name, save_dir="/kaggle/working"):
    zip_path = os.path.join(save_dir, f"{model_name}.zip")
    model_path = os.path.join(save_dir, f"{model_name}.pth")

    # Unzip the model
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(save_dir)

    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model file not found after extraction: {model_path}")

    # Load model weights
    model.load_state_dict(torch.load(model_path, map_location='cpu'))

    os.remove(model_path)

    return model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

results_log = []
total_start_time = time.time()

for i, config in enumerate(experiments):
    print(f"\n{'='*30} Experiment {i+1+20}/{len(experiments)}: {config['run_name']} {'='*30}")
    
    experiment_start_time = time.time()
    
    if config["model_type"] == "RNN":
        model = AttentiveRNN(
            input_size=len(features),
            hidden_size=config["hidden_size"],
            num_layers=config["num_layers"],
            num_classes=len(class_names),
            attention=config["attention"],
            bidirectional=config["bidirectional"]
        )
    else: # LSTM
        model = AttentiveLSTM(
            input_size=len(features),
            hidden_size=config["hidden_size"],
            num_classes=len(class_names),
            num_layers=config["num_layers"],
            attention=config["attention"],
            bidirectional=config["bidirectional"]
        )
    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs via DataParallel!")
        model = nn.DataParallel(model)
        
    model = unzip_and_load_model(model, config['run_name'])
    model.to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"])
    criterion = nn.CrossEntropyLoss()
    
    print(f"Training model: {config['run_name']} for {config['epochs']} epochs...")
    training(model, device, criterion, optimizer, train, train_epochs=config["epochs"])
    
    print(f"Evaluating model: {config['run_name']}...")
    eval_metrics = eval_detailed(model, device, dev, report_title=config['run_name'])
    
    experiment_time = time.time() - experiment_start_time
    save_model_as_zip(model, model_name=config['run_name'])
    
    run_result = {
        "run_name": config["run_name"],
        "model_type": config["model_type"],
        "num_layers": config["num_layers"],
        "hidden_size": config["hidden_size"],
        "attention": str(config["attention"]),
        "bidirectional": config["bidirectional"],
        **eval_metrics,
        "training_time_min": experiment_time / 60
    }
    results_log.append(run_result)
    
    pd.DataFrame(results_log).to_csv("experiment_report_intermediate.csv", index=False)


print(f"\n{'='*30} FINAL EXPERIMENT REPORT {'='*30}")
total_run_time = time.time() - total_start_time
print(f"Total experiment time: {total_run_time/60:.2f} minutes")

results_df = pd.DataFrame(results_log)
results_df = results_df.sort_values(by="mcc", ascending=False).reset_index(drop=True)

pd.set_option('display.float_format', '{:.4f}'.format)
display(results_df)

results_df.to_csv("experiment_report_final2.csv", index=False)

In [32]:
!zip -r Unihead.zip /kaggle/working/

  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/RNN_Uni_dot.zip (stored 0%)
  adding: kaggle/working/LSTM_Bi_NoAtt.zip (stored 0%)
  adding: kaggle/working/.virtual_documents/ (stored 0%)
  adding: kaggle/working/RNN_Bi_concat.zip (stored 0%)
  adding: kaggle/working/experiment_report_final.csv (deflated 50%)
  adding: kaggle/working/LSTM_Uni_dot.zip (stored 0%)
  adding: kaggle/working/LSTM_Bi_concat.zip (stored 0%)
  adding: kaggle/working/LSTM_Bi_dot.zip (stored 0%)
  adding: kaggle/working/RNN_Bi_NoAtt.zip (stored 0%)
  adding: kaggle/working/RNN_Bi_dot.zip (stored 0%)
  adding: kaggle/working/RNN_Uni_concat.zip (stored 0%)
  adding: kaggle/working/experiment_report_intermediate.csv (deflated 42%)
  adding: kaggle/working/RNN_Uni_NoAtt.zip (stored 0%)
  adding: kaggle/working/scaler_params.csv (deflated 57%)
  adding: kaggle/working/LSTM_Uni_NoAtt.zip (stored 0%)
  adding: kaggle/working/LSTM_Uni_concat.zip (stored 0%)


# Multi Head Attention Bi-LSTM

In [26]:
class MultiHeadBiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, num_heads=4):
        super().__init__()

        self.hidden_size = hidden_size
        self.num_heads = num_heads

        # BiLSTM
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True
        )

        # Multi-Head Attention
        self.mha = nn.MultiheadAttention(
            embed_dim=hidden_size * 2,
            num_heads=num_heads,
            batch_first=True
        )

        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, X):
        # X: (batch_size, seq_len, input_size)
        lstm_out, (h_n, c_n) = self.lstm(X)
        # lstm_out: (batch_size, seq_len, hidden_size * num_directions)

        attn_out, attn_weights = self.mha(lstm_out, lstm_out, lstm_out)
        # attn_out: (batch_size, seq_len, hidden_size * num_directions)

        context = torch.mean(attn_out, dim=1)
        # context: (batch_size, hidden_size * num_directions)

        output = self.fc(context)

        return output, attn_weights


In [31]:
experiments = []

# Experiment 1
run_name = "MH_Bi_LSTM_hs256_l2_h4"
config = {
    "run_name": run_name,
    "hidden_size": 256,
    "num_layers": 2,
    "heads": 4,
    "lr": 0.001,
    "epochs": 40
}
experiments.append(config)

# Experiment 2
run_name = "MH_Bi_LSTM_hs128_l2_h8"
config = {
    "run_name": run_name,
    "hidden_size": 128,
    "num_layers": 2,
    "heads": 8,
    "lr": 0.001,
    "epochs": 40
}
experiments.append(config)

# Experiment 3
run_name = "MH_Bi_LSTM_hs256_l3_h4"
config = {
    "run_name": run_name,
    "hidden_size": 256,
    "num_layers": 3,
    "heads": 4,
    "lr": 0.001,
    "epochs": 40
}
experiments.append(config)

# Experiment 4
run_name = "MH_Bi_LSTM_hs128_l1_h8"
config = {
    "run_name": run_name,
    "hidden_size": 128,
    "num_layers": 1,
    "heads": 8,
    "lr": 0.001,
    "epochs": 40
}
experiments.append(config)

In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

results_log = []
total_start_time = time.time()

for i, config in enumerate(experiments):
    print(f"\n{'='*30} Experiment {i+1}/{len(experiments)}: {config['run_name']} {'='*30}")
    
    experiment_start_time = time.time()
    
    
    model = MultiHeadBiLSTM(
        input_size=len(features),
        hidden_size=config["hidden_size"],
        num_classes=len(class_names),
        num_layers=config["num_layers"],
        num_heads=config["heads"]
    )
    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs via DataParallel!")
        model = nn.DataParallel(model)
        
    # model = unzip_and_load_model(model, config['run_name'])
    model.to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"])
    criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
    
    print(f"Training model: {config['run_name']} for {config['epochs']} epochs...")
    training(model, device, criterion, optimizer, train, train_epochs=config["epochs"])
    
    print(f"Evaluating model: {config['run_name']}...")
    eval_metrics = eval_detailed(model, device, dev, report_title=config['run_name'])
    
    experiment_time = time.time() - experiment_start_time
    save_model_as_zip(model, model_name=config['run_name'])
    
    run_result = {
        "run_name": config["run_name"],
        "num_layers": config["num_layers"],
        "hidden_size": config["hidden_size"],
        "num_heads": config["heads"],
        **eval_metrics,
        "training_time_min": experiment_time / 60
    }
    results_log.append(run_result)
    
    pd.DataFrame(results_log).to_csv("multihead_experiment_report_intermediate.csv", index=False)


print(f"\n{'='*30} FINAL EXPERIMENT REPORT {'='*30}")
total_run_time = time.time() - total_start_time
print(f"Total experiment time: {total_run_time/60:.2f} minutes")

results_df = pd.DataFrame(results_log)
results_df = results_df.sort_values(by="mcc", ascending=False).reset_index(drop=True)

pd.set_option('display.float_format', '{:.4f}'.format)
display(results_df)

results_df.to_csv("multihead_experiment_report.csv", index=False)


============================== Experiment 1/4: MH_Bi_LSTM_hs256_l2_h4 ==============================
Using 2 GPUs via DataParallel!
Training model: MH_Bi_LSTM_hs256_l2_h4 for 40 epochs...
Starting training for 40 epochs on cuda.


Training Progress:   0%|          | 0/40 [00:00<?, ?epoch/s]

Epoch 1 Summary -> Avg. Loss: 0.8657, Accuracy: 74.97%
Epoch 2 Summary -> Avg. Loss: 0.8290, Accuracy: 75.41%
Epoch 3 Summary -> Avg. Loss: 0.7998, Accuracy: 76.49%
Epoch 4 Summary -> Avg. Loss: 0.7778, Accuracy: 76.68%
Epoch 5 Summary -> Avg. Loss: 0.7743, Accuracy: 76.38%
Epoch 6 Summary -> Avg. Loss: 0.7523, Accuracy: 76.35%
Epoch 7 Summary -> Avg. Loss: 0.7386, Accuracy: 76.39%
Epoch 8 Summary -> Avg. Loss: 0.7222, Accuracy: 77.07%
Epoch 9 Summary -> Avg. Loss: 0.7040, Accuracy: 76.99%
Epoch 10 Summary -> Avg. Loss: 0.6897, Accuracy: 76.84%
Epoch 11 Summary -> Avg. Loss: 0.6700, Accuracy: 77.21%
Epoch 12 Summary -> Avg. Loss: 0.6474, Accuracy: 77.90%
Epoch 13 Summary -> Avg. Loss: 0.6274, Accuracy: 78.22%
Epoch 14 Summary -> Avg. Loss: 0.6209, Accuracy: 78.48%
Epoch 15 Summary -> Avg. Loss: 0.5995, Accuracy: 78.90%
Epoch 16 Summary -> Avg. Loss: 0.5893, Accuracy: 78.76%
Epoch 17 Summary -> Avg. Loss: 0.5701, Accuracy: 79.25%
Epoch 18 Summary -> Avg. Loss: 0.5571, Accuracy: 79.54%
E

Evaluating (MH_Bi_LSTM_hs256_l2_h4):   0%|          | 0/34 [00:00<?, ?it/s]


================= MH_Bi_LSTM_hs256_l2_h4 =================
Classification Report:

              precision    recall  f1-score   support

    No Flare     0.9839    0.8115    0.8894      1878
     C-Class     0.3521    0.7143    0.4717       210
     M-Class     0.2153    0.7949    0.3388        39
     X-Class     0.3333    1.0000    0.5000         4

    accuracy                         0.8020      2131
   macro avg     0.4711    0.8302    0.5500      2131
weighted avg     0.9063    0.8020    0.8374      2131


Normalized Confusion Matrix:
 [[0.8115016  0.14430245 0.04313099 0.00106496]
 [0.11904762 0.71428571 0.15238095 0.01428571]
 [0.         0.12820513 0.79487179 0.07692308]
 [0.         0.         0.         1.        ]]

Key Metrics Summary:
  balanced_accuracy   : 0.8302
  roc_auc_weighted    : 0.9251
  mcc                 : 0.4653
  f1_macro            : 0.5500
  f1_weighted         : 0.8374
MH_Bi_LSTM_hs256_l2_h4.zip saved at /kaggle/working/MH_Bi_LSTM_hs256_l2_h4.zip | Siz

...

Training Progress:   0%|          | 0/40 [00:00<?, ?epoch/s]

Epoch 1 Summary -> Avg. Loss: 0.8437, Accuracy: 75.39%
Epoch 2 Summary -> Avg. Loss: 0.8058, Accuracy: 75.87%
Epoch 3 Summary -> Avg. Loss: 0.7956, Accuracy: 76.07%
Epoch 4 Summary -> Avg. Loss: 0.7585, Accuracy: 76.18%
Epoch 5 Summary -> Avg. Loss: 0.7309, Accuracy: 76.96%
Epoch 6 Summary -> Avg. Loss: 0.7294, Accuracy: 77.12%
Epoch 7 Summary -> Avg. Loss: 0.7146, Accuracy: 77.53%
Epoch 8 Summary -> Avg. Loss: 0.7026, Accuracy: 77.85%
Epoch 9 Summary -> Avg. Loss: 0.6824, Accuracy: 78.03%
Epoch 10 Summary -> Avg. Loss: 0.6674, Accuracy: 78.24%
Epoch 11 Summary -> Avg. Loss: 0.6504, Accuracy: 78.72%
Epoch 12 Summary -> Avg. Loss: 0.6666, Accuracy: 78.64%
Epoch 13 Summary -> Avg. Loss: 0.6438, Accuracy: 78.56%
Epoch 14 Summary -> Avg. Loss: 0.6285, Accuracy: 79.07%
Epoch 15 Summary -> Avg. Loss: 0.6261, Accuracy: 78.67%
Epoch 16 Summary -> Avg. Loss: 0.6062, Accuracy: 79.34%
Epoch 17 Summary -> Avg. Loss: 0.5959, Accuracy: 79.21%
Epoch 18 Summary -> Avg. Loss: 0.5815, Accuracy: 79.56%
E

Evaluating (MH_Bi_LSTM_hs128_l1_h8):   0%|          | 0/34 [00:00<?, ?it/s]


================= MH_Bi_LSTM_hs128_l1_h8 =================
Classification Report:

              precision    recall  f1-score   support

    No Flare     0.9744    0.8099    0.8846      1878
     C-Class     0.2900    0.6476    0.4006       210
     M-Class     0.1860    0.4103    0.2560        39
     X-Class     0.2000    0.7500    0.3158         4

    accuracy                         0.7865      2131
   macro avg     0.4126    0.6544    0.4642      2131
weighted avg     0.8911    0.7865    0.8243      2131


Normalized Confusion Matrix:
 [[0.80990415 0.16719915 0.02129925 0.00159744]
 [0.18571429 0.64761905 0.13809524 0.02857143]
 [0.02564103 0.48717949 0.41025641 0.07692308]
 [0.         0.         0.25       0.75      ]]

Key Metrics Summary:
  balanced_accuracy   : 0.6544
  roc_auc_weighted    : 0.9059
  mcc                 : 0.3991
  f1_macro            : 0.4642
  f1_weighted         : 0.8243
MH_Bi_LSTM_hs128_l1_h8.zip saved at /kaggle/working/MH_Bi_LSTM_hs128_l1_h8.zip | Siz

,run_name,num_layers,hidden_size,num_heads,balanced_accuracy,roc_auc_weighted,mcc,f1_macro,f1_weighted,training_time_min
0,MH_Bi_LSTM_hs128_l2_h8,2,128,8,0.8537,0.9635,0.6430,0.7136,0.8999,67.1872
1,MH_Bi_LSTM_hs256_l3_h4,3,256,4,0.8593,0.9561,0.6084,0.6674,0.8961,243.2759
2,MH_Bi_LSTM_hs256_l2_h4,2,256,4,0.8302,0.9251,0.4653,0.5500,0.8374,173.4759
3,MH_Bi_LSTM_hs128_l1_h8,1,128,8,0.6544,0.9059,0.3991,0.4642,0.8243,46.8569


# Final Evaluation of Best Model

In [50]:
model = AttentiveLSTM(
            input_size=len(features),
            hidden_size=128,
            num_classes=len(class_names),
            num_layers=2,
            attention="dot",
            bidirectional=True
        )
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.nn.DataParallel(model) 
model_path = "/kaggle/input/lstm-bi-dot/pytorch/default/1/LSTM_Bi_dot.pth"
model.load_state_dict(torch.load(model_path, map_location='cpu'))
model.to(device)
eval_metrics = eval_detailed(model, device, test, report_title="LSTM_Bi_dot")

Evaluating (LSTM_Bi_dot):   0%|          | 0/34 [00:00<?, ?it/s]


================= LSTM_Bi_dot =================
Classification Report:

              precision    recall  f1-score   support

    No Flare     0.9902    0.9169    0.9522      1878
     C-Class     0.5770    0.9095    0.7061       210
     M-Class     0.6296    0.8718    0.7312        39
     X-Class     0.5714    1.0000    0.7273         4

    accuracy                         0.9155      2131
   macro avg     0.6921    0.9246    0.7792      2131
weighted avg     0.9421    0.9155    0.9235      2131


Normalized Confusion Matrix:
 [[0.91693291 0.07348243 0.00958466 0.        ]
 [0.07619048 0.90952381 0.00952381 0.0047619 ]
 [0.02564103 0.05128205 0.87179487 0.05128205]
 [0.         0.         0.         1.        ]]

Key Metrics Summary:
  balanced_accuracy   : 0.9246
  roc_auc_weighted    : 0.9730
  mcc                 : 0.7031
  f1_macro            : 0.7792
  f1_weighted         : 0.9235
